In [1]:
# Import all necessary libraries

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
# Import data from Yahoo Finance

symbol = 'JPM' # Example: JPMorgan Chase & Co.
df = yf.download(symbol, start='2022-01-01', end='2025-01-01')['Close']
df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Ticker,JPM
Date,
2022-01-03,146.995804
2022-01-04,152.568375
2022-01-05,149.779144
2022-01-06,151.370392
2022-01-07,152.870193
...,...
2024-12-24,239.589218
2024-12-26,240.409912
2024-12-27,238.462036


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 753 entries, 2022-01-03 to 2024-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   JPM     753 non-null    float64
dtypes: float64(1)
memory usage: 11.8 KB


In [4]:
df.isnull().sum()  

Ticker
JPM    0
dtype: int64

#### ARCH Model

In [5]:
# Import ARCH library for volatility modeling

from arch import arch_model

In [6]:
# Calculate daily returns

returns = df.pct_change().dropna()*100
returns

Ticker,JPM
Date,
2022-01-04,3.790973
2022-01-05,-1.828184
2022-01-06,1.062396
2022-01-07,0.990816
2022-01-10,0.095713
...,...
2024-12-24,1.644361
2024-12-26,0.342542
2024-12-27,-0.810231


In [7]:
# Create, fit and train the ARCH model
model = arch_model(returns, vol='ARCH', p=1)
results = model.fit(disp='off')

# Print the summary of the model
print(results.summary())

                      Constant Mean - ARCH Model Results                      
Dep. Variable:                    JPM   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -1400.95
Distribution:                  Normal   AIC:                           2807.90
Method:            Maximum Likelihood   BIC:                           2821.76
                                        No. Observations:                  752
Date:                Thu, Jun 05 2025   Df Residuals:                      751
Time:                        19:21:44   Df Model:                            1
                                 Mean Model                                
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0978  5.518e-02      1.772  7.646e-02 [-1.03

In [8]:
# Forecast volatility for the next 5 days

forecast = results.forecast(horizon=5)
predicted_volatility = forecast.variance**0.5
predicted_volatility

,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.479267,1.567022,1.577429,1.578698,1.578853


In [9]:
# Get data and calculate realized volatility

last_date = pd.to_datetime('2024-12-31')
start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09') # Buffer for weekends and holidays

realized_data = yf.download(symbol, start=start_date, end=end_date)['Close']
print(realized_data)

[*********************100%***********************]  1 of 1 completed

Ticker             JPM
Date                  
2024-12-31  237.018433
2025-01-02  237.305161
2025-01-03  240.548340
2025-01-06  239.375565
2025-01-07  241.681351
2025-01-08  241.641602


In [10]:
# Calculate Realized Returns

realized_returns = realized_data.pct_change().dropna() * 100
print(realized_returns)

Ticker           JPM
Date                
2025-01-02  0.120973
2025-01-03  1.366670
2025-01-06 -0.487542
2025-01-07  0.963250
2025-01-08 -0.016447


In [11]:
# Calculate Realized Volatility

realized_volatility = realized_returns.std()*np.sqrt(5)  # Annualized volatility
print(realized_volatility)

Ticker
JPM    1.69195
dtype: float64


In [12]:
# Calculate Average of Predicted Volatility
average_predicted_volatility = predicted_volatility.values.mean()
print(average_predicted_volatility)

1.5562537932154534


In [13]:
# Compare Realized and Predicted Volatility

print('ARCH Model Predicted Volatility:', average_predicted_volatility)
print("-----------------------------------------------------")
print('ARCH Model Realized Volatility:', realized_volatility.values[0])

ARCH Model Predicted Volatility: 1.5562537932154534
-----------------------------------------------------
ARCH Model Realized Volatility: 1.6919495467179895


#### GARCH Model

In [14]:
# Calculate daily returns

returns = df.pct_change().dropna()*100
returns.head()

Ticker,JPM
Date,
2022-01-04,3.790973
2022-01-05,-1.828184
2022-01-06,1.062396
2022-01-07,0.990816
2022-01-10,0.095713


In [15]:
# Create, fit and train the GARCH model
model = arch_model(returns, vol='GARCH', p=1, q=1)
results = model.fit(disp='off')

# Print the summary of the model
print(results.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                    JPM   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1387.47
Distribution:                  Normal   AIC:                           2782.94
Method:            Maximum Likelihood   BIC:                           2801.43
                                        No. Observations:                  752
Date:                Thu, Jun 05 2025   Df Residuals:                      751
Time:                        19:21:44   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1146  5.754e-02      1.991  4.648e-02 [1.786e-0

In [16]:
# Forecast volatility for the next 5 days

forecast = results.forecast(horizon=5)
predicted_volatility = forecast.variance**0.5
predicted_volatility

,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-31,1.623483,1.622252,1.621032,1.619824,1.618626


In [17]:
# Get data and calculate realized volatility

last_date = pd.to_datetime('2024-12-31')
start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09') # Buffer for weekends and holidays

realized_data = yf.download(symbol, start=start_date, end=end_date)['Close']
print(realized_data)

[*********************100%***********************]  1 of 1 completed

Ticker             JPM
Date                  
2024-12-31  237.018433
2025-01-02  237.305161
2025-01-03  240.548340
2025-01-06  239.375565
2025-01-07  241.681351
2025-01-08  241.641602


In [18]:
# Calculate Realized Returns

realized_returns = realized_data.pct_change().dropna() * 100
print(realized_returns)

Ticker           JPM
Date                
2025-01-02  0.120973
2025-01-03  1.366670
2025-01-06 -0.487542
2025-01-07  0.963250
2025-01-08 -0.016447


In [19]:
# Calculate Realized Volatility

realized_volatility = realized_returns.std()*np.sqrt(5)  # Annualized volatility
print(realized_volatility)

Ticker
JPM    1.69195
dtype: float64


In [20]:
# Calculate Average of Predicted Volatility
average_predicted_volatility = predicted_volatility.values.mean()
print(average_predicted_volatility)

1.6210435976782018


In [21]:
# Compare Realized and Predicted Volatility

print('GARCH Model Predicted Volatility:', average_predicted_volatility)
print("-----------------------------------------------------")
print('GARCH Model Realized Volatility:', realized_volatility.values[0])

GARCH Model Predicted Volatility: 1.6210435976782018
-----------------------------------------------------
GARCH Model Realized Volatility: 1.6919495467179895


#### EWMA Model

In [22]:
# Calculate daily returns

returns = df.pct_change().dropna()*100
returns.head()

Ticker,JPM
Date,
2022-01-04,3.790973
2022-01-05,-1.828184
2022-01-06,1.062396
2022-01-07,0.990816
2022-01-10,0.095713


In [23]:
# Ensure returns is a Series (avoid future warnings)
returns = returns.squeeze()

# Define the lambda parameter for EWMA
lambda_ = 0.94  # Commonly used value for financial time series

# Initialize EWMA variance array
ewma_var = np.zeros(len(returns))

# Set initial variance using first 5 return values
initial_variance = float(returns.iloc[:5].var(ddof=0))
ewma_var[0] = initial_variance

# Calculate EWMA variance recursively
for t in range(1, len(returns)):
    ewma_var[t] = lambda_ * ewma_var[t - 1] + (1 - lambda_) * float(returns.iloc[t - 1])**2

# Convert variance to volatility
ewma_volatility = np.sqrt(ewma_var)

# Predict volatility for t+1
next_volatility = np.sqrt(lambda_ * ewma_var[-1] + (1 - lambda_) * float(returns.iloc[-1])**2)

# Output
print(f"EWMA Model Predicted Volatility: {next_volatility:.4f}")
print("-----------------------------------------------------")
print(f"EWMA Model Realized Volatility: {ewma_volatility[-1]:.4f}")



EWMA Model Predicted Volatility: 1.4840
-----------------------------------------------------
EWMA Model Realized Volatility: 1.5301
